<a href="https://colab.research.google.com/github/lukaszlewickii/football_games_predictor/blob/main/metatreelib_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install metatreelib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s 

In [20]:
pip install imodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.2/237.2 kB 3.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import torch

In [22]:
import sys
sys.path.append('..')

from metatree.model_metatree import LlamaForMetaTree as MetaTree
from metatree.decision_tree_class import DecisionTree, DecisionTreeForest
from metatree.run_train import preprocess_dimension_patch
from transformers import AutoConfig

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import imodels
import random

model_name_or_path = "yzhuang/MetaTree"

config = AutoConfig.from_pretrained(model_name_or_path)
model = MetaTree.from_pretrained(
    model_name_or_path,
    config=config,
)
decision_tree_forest = DecisionTreeForest()

ensemble_size = 1
seed = 42

In [5]:
data = pd.read_csv('/content/england-premier-league-matches-2022-to-2023-stats.csv')

In [6]:
data.shape

(380, 66)

In [11]:
data.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,result
0,1659726000,Aug 05 2022 - 7:00pm,complete,25286.0,Crystal Palace,Arsenal,Anthony Taylor,1,0.0,0.0,...,3.49,2.09,1.29,1.90,3.25,6.75,1.80,1.95,Selhurst Park (London),2
1,1659785400,Aug 06 2022 - 11:30am,complete,NaN,Fulham,Liverpool,Andy Madley,1,0.0,0.0,...,5.40,1.34,1.15,1.66,2.43,4.10,1.91,1.91,Craven Cottage (London),0
2,1659794400,Aug 06 2022 - 2:00pm,complete,NaN,AFC Bournemouth,Aston Villa,Peter Bankes,1,0.0,0.0,...,3.53,2.30,1.35,1.95,3.66,7.50,1.80,1.95,Vitality Stadium (Bournemouth- Dorset),1
3,1659794400,Aug 06 2022 - 2:00pm,complete,NaN,Leeds United,Wolverhampton Wanderers,Robert Jones,1,0.0,0.0,...,3.58,3.07,1.34,2.03,3.62,7.50,1.80,1.95,Elland Road (Leeds- West Yorkshire),1
4,1659794400,Aug 06 2022 - 2:00pm,complete,52245.0,Newcastle United,Nottingham Forest,Simon Hooper,1,0.0,0.0,...,3.76,4.97,1.35,1.94,3.73,7.50,2.00,1.75,St. James' Park (Newcastle upon Tyne),1


In [10]:
data['result'] = np.where(data['home_team_goal_count'] == data['away_team_goal_count'], 0, np.where(data['home_team_goal_count'] > data['away_team_goal_count'], 1, 2))

In [13]:
X = data.drop('result', axis=1)
y = data['result']

In [ ]:
X, y, feature_names = imodels.get_clean_dataset('fico', data_source='imodels')


print("Dataset Shapes X={}, y={}, Num of Classes={}".format(X.shape, y.shape, len(set(y))))

train_idx, test_idx = train_test_split(range(X.shape[0]), test_size=0.3, random_state=seed)

# Dimension
random.seed(seed)
feature_idx = np.random.choice(X.shape[1], 10, replace=False)
X = X[:, feature_idx]

test_X, test_y = X[test_idx], y[test_idx]

for i in range(ensemble_size):
    # Sample Train and Test Data
    random.seed(seed+i+1)
    subset_idx = random.sample(train_idx, 256)
    train_X, train_y = X[subset_idx], y[subset_idx]


    input_x = torch.tensor(train_X, dtype=torch.float32)
    input_y = torch.nn.functional.one_hot(torch.tensor(train_y)).float()

    batch = {"input_x": input_x, "input_y": input_y, "input_y_clean": input_y}
    batch = preprocess_dimension_patch(batch, n_feature=10, n_class=10)
    model.depth = 2
    outputs = model.generate_decision_tree(batch['input_x'], batch['input_y'], depth=model.depth)
    decision_tree_forest.add_tree(DecisionTree(auto_dims=outputs.metatree_dimensions, auto_thresholds=outputs.tentative_splits, input_x=batch['input_x'], input_y=batch['input_y'], depth=model.depth))

    print("Decision Tree Features: ", [x.argmax(dim=-1) for x in outputs.metatree_dimensions])
    print("Decision Tree Threasholds: ", outputs.tentative_splits)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


fetching fico from imodels
Dataset Shapes X=(10459, 23), y=(10459,), Num of Classes=2


In [ ]:
tree_pred = decision_tree_forest.predict(torch.tensor(test_X, dtype=torch.float32))

accuracy = accuracy_score(test_y, tree_pred.argmax(dim=-1).squeeze(0))
print("MetaTree Test Accuracy: ", accuracy)

In [ ]:
cart_ensemble = []

for i in range(ensemble_size):
    random.seed(seed+i+1)
    subset_idx = random.sample(train_idx, 256)
    train_X, train_y = X[subset_idx], y[subset_idx]

    clf = sklearn.tree.DecisionTreeClassifier(max_depth=2, random_state=seed+i+1)
    clf.fit(train_X, train_y)
    cart_ensemble.append(clf)

overall_pred = np.zeros((test_X.shape[0], len(set(test_y))))
for clf in cart_ensemble:
    overall_pred += clf.predict_proba(test_X)
overall_pred = overall_pred / len(cart_ensemble)

accuracy = accuracy_score(test_y, overall_pred.argmax(axis=-1))
print("CART Test Accuracy: ", accuracy)